## INF1608 - Análise Numérica - 2017.1
## Departamento de Informática - PUC-Rio 
## Prof. Hélio Lopes - lopes@inf.puc-rio.br
## http://www.inf.puc-rio.br/~lopes



## Lista 1

1) Faça a implementação da decomposição LU, cuja entrada é uma matriz A de tamanho nxn.

2) Faça a implementação das funções LUforwardsub e da LUbackwardsub, cuja entrada é a decomposição LU de uma matriz A de tamanho nxn.

3) Implemente a função LUsolve que resolve o sistema Ax=b, tendo em mão a deomposição A=LU.

4) Teste muito bem todos esses algoritmos!

In [77]:
# ALUNO: Rafael Rubim Cabral
# MATRÍCULA: 1511068
#1)
# A função LUDecomp recebe uma matriz quadrada qualquer A e faz sua decomposição LU, sem
# considerar pivôs = 0. Retorna uma matriz LU cujos elementos da diagonal(inclusive) para cima,
# representam a matriz U e os da diagonal(exclusive) para baixo, representam a matriz L. Os
# elementos da diagonal de L (1's) são omitidos.
def LUDecomp(A):
    n = len(A)
    
    # A matriz LU é inicialmente uma cópia de A:
    LU = A[:]
    for i in range(n):
        LU[i] = A[i][:]
    
    for j in range(n):
        for i in range(j+1, n):
            LU[i][j] /= LU[j][j] # fator que multiplica linha, já trocado de sinal (matriz L)
            for k in range(j+1, n):
                LU[i][k] -= LU[i][j]*LU[j][k] # atualizar elementos da matriz U
    return LU

# A função getU recebe a matriz resultado de um LUDecomp e retorna a matriz U
def getU(LU):
    # A matriz U é inicialmente uma cópia de LU:
    U = LU[:]
    for i in range(len(LU)):
        U[i] = LU[i][:]
    
    for i in range(len(LU)):
        for j in range(i):
            U[i][j] = 0 # zera tudo abaixo da diagonal
    return U

# A função getL recebe a matriz resultado de um LUDecomp e retorna a matriz L
def getL(LU):
    # A matriz L é inicialmente uma cópia de LU:
    L = LU[:]
    for i in range(len(LU)):
        L[i] = LU[i][:]
    
    for i in range(len(LU)):
        L[i][i] = 1 # coloca 1's na diagonal
        for j in range(i+1, len(LU)):
            L[i][j] = 0 # zera tudo acima da diagonal
    return L

A = [[1, 2, 3], \
     [4, 5, 6], \
     [7, 8, 9]]
print "A = " + str(A)
LU = LUDecomp(A)
print "LUDecomp(A) = " + str(RES)
U = getU(LU)
print "U = " + str(U)
L = getL(LU)
print "L = " + str(L)
print "Eba, tudo funciona!"

A = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
LUDecomp(A) = [[1, 2, 3], [4, -3, -6], [7, 2, 0]]
U = [[1, 2, 3], [0, -3, -6], [0, 0, 0]]
L = [[1, 0, 0], [4, 1, 0], [7, 2, 1]]
Eba, tudo funciona!


In [76]:
#2)
# A função LUForwardSub recebe uma matriz (originalmente A) decomposta em LU e um vetor lado
# direito "b" da equação Ax = b. Essa função retorna um vetor y tal que Ly = b, Ux = y
# Essa função assume que a solução de Ly = b é única.
def LUForwardSub(LU, b):
    n = len(LU)
    
    # y é inicialmente uma cópia de b
    y = b[:]
    
    for i in range(n):
        
        lSum = 0 # fazer substituições dos valores já calculados
        for j in range(i):
            lSum += LU[i][j]*y[j]
        
        # o próximo valor é calculado pelas substituições na próxima equação
        y[i] = b[i] - lSum
    return y

# A função LUBackwardSub recebe uma matriz (originalmente A) decomposta em LU e um vetor "y"
# retornado por LUForwardSub. Essa função retorna um vetor x tal que Ux = y, garantindo assim
# que Ax = b. Essa função assume que a solução de Ux = y é única.
def LUBackwardSub(LU, y):
    n = len(LU)
    
    # x é inicialmente uma cópia de y
    x = y[:]
    
    for k in range(n):
        i = n-1-k # i é a linha atual tratada na matriz U
        
        uSum = 0 # fazer substituições dos valores já calculados
        for j in range(i+1, n):
            uSum += LU[i][j]*x[j]
        
        # o próximo valor é calculado pelas substituições na próxima equação
        x[i] = y[i] - uSum
        x[i] /= LU[i][i]
    return x

A = [[1, 2, 3], \
     [4, 5, 6], \
     [7, 8, 11]]
print "A = " + str(A)
LU = LUDecomp(A)
print "LUDecomp(A) = " + str(LU)
b = [5, 5, 7]
print "b = " + str(b)
y = LUForwardSub(LU,b)
print "Ly = b => y = " + str(y)
x = LUBackwardSub(LU,y)
print "Ux = y => x = " + str(x)
print "Nossa, funcionou de novo!"

A = [[1, 2, 3], [4, 5, 6], [7, 8, 11]]
LUDecomp(A) = [[1, 2, 3], [4, -3, -6], [7, 2, 2]]
b = [5, 5, 7]
Ly = b => y = [5, -15, 2]
Ux = y => x = [-4, 3, 1]
Nossa, funcionou de novo!


In [75]:
#3)
# A função LUSolve recebe uma matriz quadrada A e um vetor b e resolve o sistema Ax = b,
# retornando x. Note que o sistema deve conter solução única e o escalonamento de A não pode
# requerer uma troca de linhas para que a função funcione.
def LUSolve(A, b):
    LU = LUDecomp(A) # Decompor em LU
    y = LUForwardSub(LU, b) # Calcular y tal que Ly = b
    x = LUBackwardSub(LU, y) # Calcular x tal que Ux = y
    return x # Ly = b, Ux = y => LUx = b => Ax = b!

A = [[2., 4., 8.], [3., 7., 12.], [9., 1., 16.]]
b = [1., 2., 3.]
x = LUSolve(A, b)
print "x = " + str(x)
print "A matemática é incrível!"

x = [0.8999999999999999, 0.5, -0.35]
A matemática é incrível!


In [ ]:
#4)
# Os algoritmos foram testados no desenvolvimento das questões anteriores, e os resultados
# comparados com contas feitas à mão, que validou seus resultados. Deve-se notar, porém, que os
# resultados não teriam sido positivos para sistemas lineares com infinitas ou nenhuma solução
# ou para matrizes cujo escalonamento exige a operação de troca de linhas, pois o algoritmo
# simplificado implementado não considera nenhuma dessas possibilidades.